ปฏิบัติการครั้งที่ 3 กระบวนวิชา 229351 Statistical Learning for Data Science


###ในปฏิบัติการนี้เราจะฝึกการทำ PCA ด้วยสองวิธี คือ
1. หาด้วยการแยกส่วนประกอบของเมทริกซ์ผ่าน numpy
2. หาด้วยการใช้เครื่องมือที่มีมาให้ใน scikit-learn

####ตัวอย่าง 1: แยกตัวประกอบ (spectral decomposition) ของเมทริกซ์ต่อไปนี้

$$
M=\begin{pmatrix} 3 & 4 & 6 \\ 2 & 4 & 6 \\ 7 & 8 & 3 \end{pmatrix}$$

In [ ]:
import numpy as np

# เริ่มจากตัวอย่าง 3x3 เมทริกซ์

M = np.array([[3,4,6],[2,4,6],[7,8,3]])

M = M-np.mean(M, axis = 0)  #normalize the columns

# np.cov คำนวณ covariance ระหว่างเวกเตอร์แถว แต่เราต้องการ covariance ของ
# เวกเตอร์คอลัมน์ของ A จึงต้องทำการ tranpose ก่อนที่จะใส่ np.cov

A = np.cov(M, rowvar=False) #covariance matrix of M

print(M, '<- M\n\n')
print(A, '<- A')

####Matrix decomposition

$$A =\begin{pmatrix}\uparrow & \uparrow & \uparrow \\
u_1 & u_2 & u_3 \\
\downarrow & \downarrow & \downarrow
\end{pmatrix}\begin{pmatrix}\lambda_1 & 0 & 0 \\
0 & \lambda_2 & 0 \\
0 & 0 & \lambda_3
\end{pmatrix}\begin{pmatrix}\leftarrow & u_1 & \rightarrow \\
\leftarrow & u_2 & \rightarrow \\
\leftarrow & u_3 & \rightarrow
\end{pmatrix}$$ 

In [ ]:
#หา eigenvalues และ eigenvectors ของ A ด้วยฟังก์ชันต่อไปนี้

specA = np.linalg.eigh(A)

specA

####SpecA มีสมาชิก 2 ตัว  
ตัวที่ 1 คือ Eigenvalues  เรียงจาก**น้อยไปหามาก**  
ตัวที่สองคือ Eigenvectors โดยที่ **แต่ละ column คือ eigenvector แต่ละตัว**

\begin{align*}
\texttt{specA[0]} &=  D=[\lambda_1,\lambda_2,\lambda_3] \\
\texttt{specA[1]} &=  U=\begin{pmatrix}\uparrow & \uparrow & \uparrow \\
u_1 & u_2 & u_3 \\
\downarrow & \downarrow & \downarrow
\end{pmatrix}
\end{align*}

In [ ]:
D = specA[0]
U = specA[1]

print(D)
print(U)

In [ ]:
#สร้าง diagonal matrix ทีมี eigenvalues ในแนวแทยง

diagD = np.diag(D)

print(diagD)

####ลองเช็คว่าการแยกส่วนประกอบของ A ถูกต้องหรือไม่ดัวยสูตร

$$A =\begin{pmatrix}\uparrow & \uparrow & \uparrow \\
u_1 & u_2 & u_3 \\
\downarrow & \downarrow & \downarrow
\end{pmatrix}\begin{pmatrix}\lambda_1 & 0 & 0 \\
0 & \lambda_2 & 0 \\
0 & 0 & \lambda_3
\end{pmatrix}\begin{pmatrix}\leftarrow & u_1 & \rightarrow \\
\leftarrow & u_2 & \rightarrow \\
\leftarrow & u_3 & \rightarrow
\end{pmatrix}$$ 

In [ ]:
print(A, ' <- A')

print(U @ diagD @ U.T,' <- U^TDU')

####ทำ PCA จากข้อมูลที่มีอยู่ 3 มิติให้เหลือ 2 มิติ
1. เริ่มด้วยการหา Eigenvector ที่มีค่า Eigenvalue สูงสุดสองอันดับแรก

In [ ]:
# D = [0, 0.137, 15.2]

l_2 = D[-1] #eigenvalue ที่มีค่าสูงที่สุด
l_3 = D[-2] #eigenvalue ที่มีค่ารองลงมา

U_23 = U[:,-2:]


print(U_23, ' <- U_2')

####ให้ตัวเลขในแต่ละแถวของ $A$ ทำ projection บนเวกเตอร์ $u_1$ และ $u_2$ ทำได้โดย
$$MU_{23}=\begin{pmatrix} \longleftarrow & x & \longrightarrow \\ \longleftarrow & y & \longrightarrow \\ \longleftarrow & z & \longrightarrow \end{pmatrix}\begin{pmatrix}\uparrow & \uparrow \\
u_2 & u_3  \\
\downarrow & \downarrow 
\end{pmatrix}=\begin{pmatrix}x\cdot u_2 &x\cdot u_3 \\
y\cdot u_2 &y\cdot u_3 \\
z\cdot u_2 &z\cdot u_3\end{pmatrix}$$

In [ ]:
M2 = M @ U_23 #M คือ matrix ที่มีข้อมูลอยู่
print(M2)

In [ ]:
import matplotlib.pyplot as plt

# Plot initialisation
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(M[:,0], M[:,1], M[:,2],  cmap="Set2_r", s=60);

ax2 = fig.add_subplot(122)
ax2.scatter(M2[:,0], M2[:,1],  cmap="Set2_r", s=60);

####การทำ PCA ด้วย scikit-learn สามารถทำได้ดังนี้

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
B2 = pca.fit_transform(M)
print(B2)

####ลองดูว่าทิศทางของ principal component มีอะไรบ้าง

In [ ]:
print(U_23)

pca.components_

####สามารถดูได้ว่าในแต่ละทิศได้อธิบายความแปรปรวนของข้อมูลไปเท่าไหร่

In [ ]:
pca.explained_variance_

####หรือสามารถดูเป็นสัดส่วนได้

In [ ]:
pca.explained_variance_ratio_